In [2]:


import csv

import numpy as np
import pysolr
from tqdm import tqdm

from config import TLCPaths
from preprocessing import cistem


import csv

import numpy as np
import pysolr
from tqdm import tqdm

from config import TLCPaths
from preprocessing import cistem

In [3]:
solr = pysolr.Solr('http://localhost:8983/solr/wumls-single-valued-deduplicated',
                   always_commit=False)
# solr = pysolr.Solr('http://localhost:8983/solr/wumls-multi-valued', always_commit=False)
solr.ping()

'{\n  "responseHeader":{\n    "zkConnected":null,\n    "status":0,\n    "QTime":4,\n    "params":{\n      "q":"{!lucene}*:*",\n      "distrib":"false",\n      "df":"_text_",\n      "rows":"10",\n      "echoParams":"all",\n      "rid":"localhost-3420"}},\n  "status":"OK"}\n'

In [4]:
def create_query_with_edit_distance(term):
    return "{!func}strdist('" + term + "',index_term,edit)"


In [5]:
search_term = "pyelonephriti"
results = solr.search(q=create_query_with_edit_distance(term=search_term), fl="*,score", rows=10)
for res in results:
    print(res)

{'cui': ['C0034186'], 'source': ['MDRGER'], 'language': ['GER'], 'name': ['Pyelonephritis'], 'index_term': ['pyelonephriti'], 'id': 'c8b1ada1-826d-46fa-a6fb-186add6196a1', '_version_': 1760194257981800490, 'score': 1.0}
{'cui': ['C0034188'], 'source': ['MDRGER'], 'language': ['GER'], 'name': ['Xanthogranulomatoese Pyelonephritis'], 'index_term': ['xanthogranulomato pyelonephriti'], 'id': 'c01e43d7-a3e5-482d-800a-73b49306f2b7', '_version_': 1760194257982849026, 'score': 1.0}
{'cui': ['C1697444'], 'source': ['MDRGER'], 'language': ['GER'], 'name': ['virale Pyelonephritis'], 'index_term': ['viral pyelonephriti'], 'id': 'c1a048cc-3de9-4231-9820-688e08951389', '_version_': 1760194263649353728, 'score': 1.0}
{'cui': ['C0034188'], 'source': ['MSHGER'], 'language': ['GER'], 'name': ['Pyelonephritis, xanthogranulomatöse'], 'index_term': ['pyelonephritis, xanthogranulomato'], 'id': '1729c369-87a4-4b8a-b12c-1953689789be', '_version_': 1760194257982849025, 'score': 0.9285714}
{'cui': ['C1328529'],

In [6]:
X, Y = [], []
# with open(TLCPaths.project_data_path.joinpath("german_mentions_and_labels.csv"),
#           newline='') as csvfile:
#     reader = csv.reader(csvfile, delimiter=' ', quotechar='|')
#     for row in reader:
#         assert len(row) == 2
#         X.append(row[0])
#         Y.append(row[1])
import json
with open(TLCPaths.project_data_path.joinpath("accepted_both_mention_and_cui_wumls_filtered.json"), "r") as f:
    data = json.load(f)
    X = data["X"]
    Y = data["Y"]
print(len(X))#%%


412


In [11]:
y_pred = []
y_pred_names = []
for mention in tqdm(X):
    stemmed_mention = cistem.stem(mention)
    results = solr.search(q=create_query_with_edit_distance(term=stemmed_mention), fl="*,score", rows=10)
    found_cuis = [res["cui"] for res in results]
    found_names = [res["name"] for res in results]
    y_pred.append(found_cuis)
    y_pred_names.append(found_names)

100%|██████████| 412/412 [03:00<00:00,  2.29it/s]


In [12]:
y_pred = np.array(y_pred)
y_pred_names = np.array(y_pred_names)

In [13]:
def top_k_acc(y_true, y_pred, k=1):
  y_pred = y_pred[:,:k]
  correct_preds = [true in pred for true,pred in zip(y_true,y_pred)]
  return np.mean(correct_preds)

for i in range(1,11):
    print(f"Acc@{i}: ", round(top_k_acc(Y,y_pred, k=i),3))

Acc@1:  0.461
Acc@2:  0.485
Acc@3:  0.507
Acc@4:  0.512
Acc@5:  0.519
Acc@6:  0.519
Acc@7:  0.524
Acc@8:  0.534
Acc@9:  0.536
Acc@10:  0.536


In [14]:
import time
timestr=time.strftime("%Y%m%d-%H%M%S")

rows = []
for mention, name, true_cui, pred_cui in zip(X, np.array(y_pred_names)[:,0], Y, y_pred[:,0]):
      rows.append([mention, name[0], true_cui, pred_cui[0], true_cui == pred_cui[0]])
rows = sorted(rows, key=lambda x: x[-1])

with open(TLCPaths.project_data_path.joinpath(f"german_tlc_prediction_SOLR_{timestr}.csv"),"w", newline="") as f:
    spamwriter = csv.writer(f, delimiter=' ',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    spamwriter.writerow(["mention", "Predicted Concept", "true cui", "pred cui", "Correct"])
    for row in rows:
      spamwriter.writerow(row)

In [ ]:
# interesting samples
# bauchspeicheldrüse |Künstliche Bauchspeicheldrüse| C0183516 C0336563 False
# Blähungen Blähung C1291077 C0016204 False
# Blähen blase C0016204 C0005758 False